In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

### Leitura de dataset

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_microdados_2023.pkl')

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

### Buscar melhorar modelo alterando Features

Tentativas de melhorar o resultado alterando as features

#### a. Tentar melhorar removendo todas as variáveis sem categorias definidas ("Não informado")

    TP_ESTADO_CIVIL: 0 (Não informado)
    TP_COR_RACA: 0 (Não declarado)
    TP_NACIONALIDADE: 0 (Não informado)
    TP_ESCOLA: 1 (Não Respondeu)
    TP_ENSINO: 0 (Não informado)

In [5]:
# Remover linhas com valores não explicativos
mascaras_validas_train = (
    (X_train['CAT_NACIONALIDADE'] != 0) &
    (X_train['CAT_ENSINO'] != 0) &
    (X_train['CAT_COR_RACA'] != 0) &
    (X_train['CAT_ESTADO_CIVIL'] != 0) &
    (X_train['CAT_ESCOLA'] != 1)
)
X_train_a = X_train[mascaras_validas_train]
y_train_a = y_train.loc[X_train_a.index]

mascaras_validas_train_final = (
    (X_train_final['CAT_NACIONALIDADE'] != 0) &
    (X_train_final['CAT_ENSINO'] != 0) &
    (X_train_final['CAT_COR_RACA'] != 0) &
    (X_train_final['CAT_ESTADO_CIVIL'] != 0) &
    (X_train_final['CAT_ESCOLA'] != 1)
)
X_train_final_a = X_train_final[mascaras_validas_train_final]
y_train_final_a = y_train_final.loc[X_train_final_a.index]

mascaras_validas_eval = (
    (X_eval['CAT_NACIONALIDADE'] != 0) &
    (X_eval['CAT_ENSINO'] != 0) &
    (X_eval['CAT_COR_RACA'] != 0) &
    (X_eval['CAT_ESTADO_CIVIL'] != 0) &
    (X_eval['CAT_ESCOLA'] != 1)
)
X_eval_a = X_eval[mascaras_validas_eval]
y_eval_a = y_eval.loc[X_eval_a.index]

X_test_a = X_test.copy()
y_test_a = y_test.copy()

In [6]:
# Criar o modelo LGBMRegressor sem categorias definidas
modelo_a = lgbm.LGBMRegressor(random_state=42,
                                max_bin=4095, 
                                force_row_wise=True)

# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (52, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 100),                         # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                   # Número de árvores
    'subsample': (0.5, 0.9),                        # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.3, 0.9),                 # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1e-1, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-6, 1e-4, 'log-uniform'),      # Regularização L2
}

# Configurar a busca Bayesiana usando BayesSearchCV
# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_a,             # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=5,                       # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
}

# Executar a busca Bayesiana
start_time = time.time()
bayes_search.fit(X_train_a, y_train_a['NUM_NOTA_CH'], **fit_params)
# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 4245
[LightGBM] [Info] Number of data points in the train set: 537944, number of used features: 40
[LightGBM] [Info] Start training from score 529.366449


In [7]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        "colsample_bytree": 0.5460623753119883,
        "learning_rate": 0.0053422688874711095,
        "max_depth": 97,
        "n_estimators": 5316,
        "num_leaves": 57,
        "reg_alpha": 0.006733444192152367,
        "reg_lambda": 5.033414197773552e-06,
        "subsample": 0.7958016936761683
    }
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.5460623753119883), ('learning_rate', 0.0053422688874711095), ('max_depth', 97), ('n_estimators', 5316), ('num_leaves', 57), ('reg_alpha', 0.006733444192152367), ('reg_lambda', 5.033414197773552e-06), ('subsample', 0.7958016936761683)])
R2:  0.3113621349059856
Tempo total de execução: 12681.03 segundos


In [8]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_a.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_a.fit(X_train_final_a, 
                y_train_final_a['NUM_NOTA_CH'], 
                eval_set=[(X_eval_a, y_eval_a['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features,
                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4285
[LightGBM] [Info] Number of data points in the train set: 457240, number of used features: 40
[LightGBM] [Info] Start training from score 529.301688
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2257]	valid_0's rmse: 69.9511	valid_0's l1: 55.1395	valid_0's l2: 4893.16


In [9]:
# Previsões
y_pred_explicativas = modelo_a.predict(X_test_a)

In [10]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_a,
                    parametros={**modelo_a.get_params(), "amostra": X_train_a.shape[0], "tempo": tempo_treino},
                    X_train=X_train_a,
                    y_train=y_train_a,
                    y_test=y_test_a,
                    y_pred=y_pred_explicativas,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV sem categorias definidas')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas' already exists. Creating a new version of this model...
2025/06/14 17:56:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seco

🏃 View run unique-colt-427 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/944f331f4a2c4cd9a13ae44641f9504a
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_remocao_de_variaveis_sem_categorias_definidas
Rastreamento do MLflow finalizado.


In [11]:
# Avaliação grupo treino
avaliar_modelo(y_train_a['NUM_NOTA_CH'], modelo_a.predict(X_train_a), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test_a['NUM_NOTA_CH'], y_pred_explicativas, "teste")

MAE (treino): 53.1030
RMSE (treino): 67.4584
R2 (treino): 0.3613
MAE (teste): 55.0613
RMSE (teste): 69.8204
R2 (teste): 0.3162


### 2. Tentar melhorar removendo variáveis de menor importância (importância e informação mútua)

    CAT_SIT_FUNC_ESC
    CAT_NACIONALIDADE
    CAT_ENSINO
    CAT_LOCALIZACAO_ESC
    CAT_ESTADO_CIVIL
    BIN_Q002_DUMMY_H
    NUM_Q017
    NUM_Q012
    BIN_Q001_DUMMY_H
    NUM_Q015

In [14]:
# Remover algumas colunas com MENOR valor explicativo
menos_importantes=['CAT_SIT_FUNC_ESC',
                    'CAT_NACIONALIDADE',
                    'CAT_ENSINO',
                    'CAT_LOCALIZACAO_ESC',
                    'CAT_ESTADO_CIVIL',
                    'BIN_Q002_DUMMY_H',
                    'NUM_Q017',
                    'NUM_Q012',
                    'BIN_Q001_DUMMY_H',
                    'NUM_Q015',
                ]

X_train_b = X_train.drop(columns=menos_importantes).copy()
y_train_b = y_train.copy()

X_train_final_b = X_train_final.drop(columns=menos_importantes).copy()
y_train_final_b = y_train_final.copy()

X_eval_b = X_eval.drop(columns=menos_importantes).copy()
y_eval_b = y_eval.copy()

X_test_b = X_test.drop(columns=menos_importantes).copy()
y_test_b = y_test.copy()

# Obter colunas categóricas
categorical_features = X_train_b.select_dtypes(include=['category']).columns.tolist()

In [15]:
# Criar o modelo LGBMRegressor sem categorias definidas
modelo_b = lgbm.LGBMRegressor(random_state=42,
                                max_bin=4095, 
                                force_row_wise=True)

# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (52, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 100),                         # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                   # Número de árvores
    'subsample': (0.5, 0.9),                        # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.3, 0.9),                 # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1e-1, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-6, 1e-4, 'log-uniform'),      # Regularização L2
}

# Configurar a busca Bayesiana usando BayesSearchCV
# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_b,             # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=5,                       # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
}

# Executar a busca Bayesiana
start_time = time.time()
bayes_search.fit(X_train_b, y_train_b['NUM_NOTA_CH'], **fit_params)
# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 4227
[LightGBM] [Info] Number of data points in the train set: 573194, number of used features: 30
[LightGBM] [Info] Start training from score 527.941622


In [16]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        "colsample_bytree":  0.5460623753119883,
        "learning_rate": 0.0053422688874711095,
        "max_depth": 97,
        "n_estimators": 5316,
        "num_leaves": 57,
        "reg_alpha": 0.006733444192152367,
        "reg_lambda": 5.033414197773552e-06,
        "subsample": 0.7958016936761683
    }
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.5460623753119883), ('learning_rate', 0.0053422688874711095), ('max_depth', 97), ('n_estimators', 5316), ('num_leaves', 57), ('reg_alpha', 0.006733444192152367), ('reg_lambda', 5.033414197773552e-06), ('subsample', 0.7958016936761683)])
R2:  0.3112070763981326
Tempo total de execução: 48518.06 segundos


In [17]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_b.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_b.fit(X_train_final_b,
                y_train_final_b['NUM_NOTA_CH'], 
                eval_set=[(X_eval_b, y_eval_b['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features,
                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4258
[LightGBM] [Info] Number of data points in the train set: 487214, number of used features: 30
[LightGBM] [Info] Start training from score 527.881246
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1995]	valid_0's rmse: 70.192	valid_0's l1: 55.3924	valid_0's l2: 4926.92


In [18]:
# Previsões
y_pred_var_importantes = modelo_b.predict(X_test_b)

In [19]:
nome_experimento = 'Notas CH ENEM 2023'

# Avaliar o modelo
registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_b,
                    parametros={**modelo_b.get_params(), "amostra": X_train_b.shape[0], "tempo": tempo_treino},
                    X_train=X_train_b,
                    y_train=y_train_b,
                    y_test=y_test_b,
                    y_pred=y_pred_var_importantes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_reducao_de_10_features',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV com redução de 10 features')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/06/15 08:32:57 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\ferra\AppData\Local\Temp\tmppix6djib\model\model.pkl, flavor: sklearn). Fall back to

🏃 View run puzzled-gnat-547 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/a0364b405b1640b3b697c0a0d12ec036
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_reducao_de_10_features
Rastreamento do MLflow finalizado.


In [20]:
# Avaliação grupo treino
avaliar_modelo(y_train_b['NUM_NOTA_CH'], modelo_b.predict(X_train_b), "treino")
# Avaliação grupo teste
avaliar_modelo(y_test_b['NUM_NOTA_CH'], y_pred_var_importantes, "teste")

MAE (treino): 53.5820
RMSE (treino): 67.9726
R2 (treino): 0.3553
MAE (teste): 55.1367
RMSE (teste): 69.9228
R2 (teste): 0.3142


### 3. Aplicar os dois: remover categorias não explicativas + remover colunas menos importantes

In [21]:
# Máscara para linhas válidas
mascara_validas = lambda df: (
    (df['CAT_NACIONALIDADE'] != 0) &
    (df['CAT_ENSINO'] != 0) &
    (df['CAT_COR_RACA'] != 0) &
    (df['CAT_ESTADO_CIVIL'] != 0) &
    (df['CAT_ESCOLA'] != 1)
)

menos_importantes = [
    'CAT_SIT_FUNC_ESC', 'CAT_NACIONALIDADE', 'CAT_ENSINO', 'CAT_LOCALIZACAO_ESC',
    'CAT_ESTADO_CIVIL', 'BIN_Q002_DUMMY_H', 'NUM_Q017', 'NUM_Q012',
    'BIN_Q001_DUMMY_H', 'NUM_Q015'
]

# Aplicar máscara e remover colunas em uma linha para cada conjunto
X_train_c = X_train[mascara_validas(X_train)].drop(columns=menos_importantes)
y_train_c = y_train.loc[X_train_c.index]

X_train_final_c = X_train_final[mascara_validas(X_train_final)].drop(columns=menos_importantes)
y_train_final_c = y_train_final.loc[X_train_final_c.index]

X_eval_c = X_eval[mascara_validas(X_eval)].drop(columns=menos_importantes)
y_eval_c = y_eval.loc[X_eval_c.index]

X_test_c = X_test.drop(columns=menos_importantes)
y_test_c = y_test.copy()

# Obter colunas categóricas
categorical_features = X_train_c.select_dtypes(include=['category']).columns.tolist()

In [22]:
# Criar o modelo LGBMRegressor sem categorias definidas
modelo_c = lgbm.LGBMRegressor(random_state=42,
                                max_bin=4095, 
                                force_row_wise=True)

# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (52, 60),                         # Número de folhas na árvore de decisão
    'max_depth': (60, 100),                         # Profundidade máxima da árvore
    'learning_rate': (0.001, 0.01, 'log-uniform'),  # Taxa de aprendizado
    'n_estimators': (5000, 6000),                   # Número de árvores
    'subsample': (0.5, 0.9),                        # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.3, 0.9),                 # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-3, 1e-1, 'log-uniform'),       # Regularização L1
    'reg_lambda': (1e-6, 1e-4, 'log-uniform'),      # Regularização L2
}

# Configurar a busca Bayesiana usando BayesSearchCV
# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_c,             # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=5,                       # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

fit_params = {
    'eval_metric': ['r2', 'rmse', 'mae'],               # Métricas a serem avaliadas
    'categorical_feature': categorical_features,        # Colunas categóricas
}

# Executar a busca Bayesiana
start_time = time.time()
bayes_search.fit(X_train_c, y_train_c['NUM_NOTA_CH'], **fit_params)
# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 4209
[LightGBM] [Info] Number of data points in the train set: 537944, number of used features: 30
[LightGBM] [Info] Start training from score 529.366449


In [23]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {
        "colsample_bytree": 0.5460623753119883,
        "learning_rate": 0.0053422688874711095,
        "max_depth": 97,
        "n_estimators": 5316,
        "num_leaves": 57,
        "reg_alpha": 0.006733444192152367,
        "reg_lambda": 5.033414197773552e-06,
        "subsample": 0.7958016936761683
    }
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.5460623753119883), ('learning_rate', 0.0053422688874711095), ('max_depth', 97), ('n_estimators', 5316), ('num_leaves', 57), ('reg_alpha', 0.006733444192152367), ('reg_lambda', 5.033414197773552e-06), ('subsample', 0.7958016936761683)])
R2:  0.3086380129697774
Tempo total de execução: 30557.31 segundos


In [24]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_c.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_c.fit(X_train_final_c,
                y_train_final_c['NUM_NOTA_CH'], 
                eval_set=[(X_eval_c, y_eval_c['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features,
                callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 4249
[LightGBM] [Info] Number of data points in the train set: 457240, number of used features: 30
[LightGBM] [Info] Start training from score 529.301688
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[2006]	valid_0's rmse: 70.0653	valid_0's l1: 55.2435	valid_0's l2: 4909.15


In [25]:
# Previsões
y_pred_var_feat_categ = modelo_c.predict(X_test_c)

In [26]:
nome_experimento = 'Notas CH ENEM 2023'

# Avaliar o modelo
registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_c,
                    parametros={**modelo_c.get_params(), "amostra": X_train_c.shape[0], "tempo": tempo_treino},
                    X_train=X_train_c,
                    y_train=y_train_c,
                    y_test=y_test_c,
                    y_pred=y_pred_var_feat_categ,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_reducao_de_10_features_categorias_nao_explicativas',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV com redução de 10 features e categorias não explicativas')

d:\Armazenamento\MBA\TCC\TccMbaUspEsalq\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
Registered model 'modelo_lgbm_reducao_de_10_features_categorias_nao_explicativas' already exists. Creating a new version of this model...
2025/06/15 17:19:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300

🏃 View run judicious-boar-636 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/28dd05e984c14bc89eac86b6d35233d6
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_reducao_de_10_features_categorias_nao_explicativas
Rastreamento do MLflow finalizado.


In [27]:
# Avaliação grupo treino
avaliar_modelo(y_train_c['NUM_NOTA_CH'], modelo_c.predict(X_train_c), "treino")
# Avaliação grupo teste
avaliar_modelo(y_test_c['NUM_NOTA_CH'], y_pred_var_feat_categ, "teste")

MAE (treino): 53.3819
RMSE (treino): 67.7830
R2 (treino): 0.3551
MAE (teste): 55.1422
RMSE (teste): 69.9457
R2 (teste): 0.3137


Conclusão

- Remoção de categorias não explicativas
    - removendo apenas as linhas com categorias não explicativas houve uma piora do modelo
    - R² de 0,3201 para 0.3162

- Remoção das 10 variáveis menos explicativas
    - removendo apenas as colunas com menor importância para o alvo houve uma piora do modelo
    - Removendo 10: R² de 0,3201 para 0,3142

- Aplicando as duas ténicas
    - Redução R² de 0,3201 para 0,3137

Padrão se mantém para outras métricas RMSE e MAE

Ambas abordagens não melhoram o modelo.